# Thermostat demo
### This notebook is an introduction to Thermostat, to both the data hub and collection of explanation data (feature attribution maps) and the accompanying convenience functions for analysis of the maps.

To start off, we have to install the dependencies.

In [1]:
!pip3 install --upgrade pip
!pip3 install cmake
!pip3 install cython
!pip3 install numpy
!pip3 install torch
!pip3 install datasets
!pip3 install spacy
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install overrides
!pip3 install jsonnet
!pip3 install sklearn
!pip3 install pandas

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\pip\\py.typed'
Consider using the `--user` option or check the permissions.



  Using cached jsonnet-0.18.0.tar.gz (592 kB)
  Running setup.py clean for jsonnet
Failed to build jsonnet
    Running setup.py install for jsonnet: started
    Running setup.py install for jsonnet: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\49176\\AppData\\Local\\Temp\\pip-install-vrsb96ad\\jsonnet_8f800cc73699425d8babbbf5b9340802\\setup.py'"'"'; __file__='"'"'C:\\Users\\49176\\AppData\\Local\\Temp\\pip-install-vrsb96ad\\jsonnet_8f800cc73699425d8babbbf5b9340802\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\49176\AppData\Local\Temp\pip-wheel-qsb54w8v'
       cwd: C:\Users\49176\AppData\Local\Temp\pip-install-vrsb96ad\jsonnet_8f800cc73699425d8babbbf5b9340802\
  Complete output (4 lines):
  running bdist_wheel
  running build
  running build_ext
  error: [WinError 2] The system cannot find the file spe

Next, we import some utilities.

In [2]:
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')


import sys
# Include root directory in module path
sys.path.append('src')

from pprint import pprint

Now we can import our library.

In [3]:
import thermostat

# Load dataset

Let's use the `load` method which is a wrapper around the `load_dataset` function from HF `datasets`.
In the background, this uses the dataset script ("hf_dataset.py") in the "thermostat" directory.

In this example, we use the `imdb-bert-lig` configuration.
This refers to **Layer Integrated Gradients** (LIG) explanations of the predictions by a **BERT** model that has been fine-tuned on the **IMDb** (train) dataset and evaluated on the **IMDb** test dataset.
In other words, we load the 25k test examples from the IMDb test plus the BERT predictions and the feature attributions from a Layer Integrated Gradients explainer.

In [10]:
lig = thermostat.load("imdb-bert-lig")

Loading Thermostat configuration: imdb-bert-lig


Downloading:   0%|          | 0.00/264M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset thermostat downloaded and prepared to C:\Users\49176\.cache\huggingface\datasets\thermostat\imdb-bert-lig\1.0.1\0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b. Subsequent calls will reuse this data.


Let's see what's inside the loaded dataset:

In [17]:
print(lig)

IMDb dataset, BERT model, Layer Integrated Gradients explanations
Dataset: imdb
Model: textattack/bert-base-uncased-imdb
Explainer: LayerIntegratedGradients



Now let's inspect a single instance of the loaded dataset.
For readability purposes, we will not print the whole content of that instance.
Instead, we're showing only the first few entries of the attributions and the input_ids, respectively.

In [18]:
example = lig[400]

print(f'Index: {example.idx}')
print(f'Attributions (first 4): {example.attributions[:4]}')
print(f'True label: {example.true_label}')
print(f'Predicted label: {example.predicted_label}')

Index: 400
Attributions (first 4): [0.0, 0.09099612385034561, -0.0480487197637558, 0.057379502803087234]
True label: pos
Predicted label: pos


We can also print a heatmap of token-attribution tuples via the `explanation` attribute of an instance.

In [19]:
pprint(example.explanation)

[('[CLS]', 0.0, 0),
 ('i', 0.09099612385034561, 1),
 ('first', -0.0480487197637558, 2),
 ('saw', 0.057379502803087234, 3),
 ('it', -0.03706467151641846, 4),
 ('at', 0.021026495844125748, 5),
 ('5', -0.008510066196322441, 6),
 ('##am', 0.012302031740546227, 7),
 ('january', 0.221795454621315, 8),
 ('1', 0.2052830308675766, 9),
 (',', 0.358757883310318, 10),
 ('2009', 0.13966944813728333, 11),
 (',', -0.003388336393982172, 12),
 ('and', 0.043109435588121414, 13),
 ('after', -0.061144351959228516, 14),
 ('a', -0.0742059275507927, 15),
 ('day', 0.09290048480033875, 16),
 ('i', 0.004284464288502932, 17),
 ('watched', -0.07977557182312012, 18),
 ('it', 0.0755855143070221, 19),
 ('again', 0.022673269733786583, 20),
 ('and', 0.041949737817049026, 21),
 ('i', 0.09280271828174591, 22),
 ('want', 0.03193451091647148, 23),
 ('to', 0.14589230716228485, 24),
 ('watch', 0.04564334452152252, 25),
 ('it', 0.06388739496469498, 26),
 ('again', -0.027509840205311775, 27),
 ('.', 0.017449360340833664, 28),

Another option is printing it as a `pandas` DataFrame which is accessible via the `.heatmap` attribute.

In [20]:
import pandas as pd
pd.set_option('display.max_columns', None)

print(example.heatmap)

token_index    0         1         2         3         4         5    \
token        [CLS]         i     first       saw        it        at   
attribution    0.0  0.170457 -0.090006  0.107485 -0.069431  0.039387   
text_field    text      text      text      text      text      text   

token_index       6         8         9         10        11        12   \
token             5am   january         1         ,      2009         ,   
attribution  0.023045  0.415474  0.384542  0.672036  0.261633 -0.006347   
text_field       text      text      text      text      text      text   

token_index       13        14        15        16        17        18   \
token             and     after         a       day         i   watched   
attribution  0.080754 -0.114537 -0.139005  0.174024  0.008026 -0.149438   
text_field       text      text      text      text      text      text   

token_index       19        20        21        22        23       24   \
token              it     again    

# Visualize data
Now the much more visually pleasing way is to turn the attribution scores into colors and display the heatmap using the displaCy (spaCy) library. We can do this with the `.render()` function.

In [16]:
example.render()

# Aggregate data
Let us first compare the heatmaps of two different models on the same dataset+explainer configuration, MNLI + Occlusion.

In [31]:
bert = thermostat.load("multi_nli-bert-occ")
electra = thermostat.load("multi_nli-electra-occ")

Reusing dataset thermostat (C:\Users\49176\.cache\huggingface\datasets\thermostat\multi_nli-bert-occ\1.0.1\0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b)


Loading Thermostat configuration: multi_nli-bert-occ


Reusing dataset thermostat (C:\Users\49176\.cache\huggingface\datasets\thermostat\multi_nli-electra-occ\1.0.1\0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b)


Loading Thermostat configuration: multi_nli-electra-occ


Now let's build a list of instances for which the predicted labels of BERT and ELECTRA do not align:

In [32]:
disagreement = [(b, e) for (b, e) in zip(bert, electra) if b.predicted_label != e.predicted_label]

We choose one instance (index: 51) and render both heatmaps:

In [33]:
u_b, u_e = disagreement[51]
print(f'Instance index: {u_b.idx}')

print(f'Model: {u_b.model_name} | Pred: {u_b.predicted_label} | True: {u_b.true_label}')
u_b.render()
print(f'Model: {u_e.model_name} | Pred: {u_e.predicted_label} | True: {u_e.true_label}')
u_e.render()

Instance index: 378
Model: textattack/bert-base-uncased-MNLI | Pred: entailment | True: contradiction


Model: howey/electra-base-mnli | Pred: contradiction | True: contradiction


We observe that the Occlusion explainer does not attribute much importance to the phrase *can be lost in an instant*. This is plausible since the heatmap explains a misclassification: the maximum output activation stands for `entailment`, but the correct label is `contradiction` and the phrase certainly is a signal for `contradiction`. In contrast, in the case of ELECTRA which correctly classified the instance the signal phrase receives much higher importance scores.

### Bonus: Print classification report from sklearn
We also added the classification report function from sklearn as a method to apply to a Thermopack.

In [24]:
for model_name, data in zip(["bert", "electra"], [bert, electra]):
    print(model_name)
    data.classification_report()
    print('=====================\n\n')

bert
               precision    recall  f1-score   support

contradiction       0.86      0.87      0.86      3213
   entailment       0.89      0.85      0.87      3479
      neutral       0.79      0.82      0.81      3123

     accuracy                           0.85      9815
    macro avg       0.85      0.85      0.85      9815
 weighted avg       0.85      0.85      0.85      9815



electra
               precision    recall  f1-score   support

   entailment       0.93      0.87      0.90      3479
      neutral       0.83      0.87      0.85      3123
contradiction       0.91      0.92      0.91      3213

     accuracy                           0.89      9815
    macro avg       0.89      0.89      0.89      9815
 weighted avg       0.89      0.89      0.89      9815





# Explainer comparison

We can only compare the heatmaps between multiple explainers. For this, let us load both the MultiNLI-BERT-Occlusion dataset plus the associated LIG and LIME explainers.

In [14]:
unit_index = 378
u_occ = thermostat.load("multi_nli-bert-occ")[unit_index]
u_intg = thermostat.load("multi_nli-bert-lig")[unit_index]
u_lime = thermostat.load("multi_nli-bert-lime")[unit_index]

Reusing dataset thermostat (C:\Users\49176\.cache\huggingface\datasets\thermostat\multi_nli-bert-occ\1.0.1\0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b)


Loading Thermostat configuration: multi_nli-bert-occ
Dataset path is D:\Working Student\repo\thermostat\src\thermostat\dataset.py
Additional parameters for loading: {}
Loading Thermostat configuration: multi_nli-bert-lig
Dataset path is D:\Working Student\repo\thermostat\src\thermostat\dataset.py
Additional parameters for loading: {}


Downloading:   0%|          | 0.00/58.5M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset thermostat downloaded and prepared to C:\Users\49176\.cache\huggingface\datasets\thermostat\multi_nli-bert-lig\1.0.1\0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b. Subsequent calls will reuse this data.
Loading Thermostat configuration: multi_nli-bert-lime
Dataset path is D:\Working Student\repo\thermostat\src\thermostat\dataset.py
Additional parameters for loading: {}


Downloading:   0%|          | 0.00/59.4M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset thermostat downloaded and prepared to C:\Users\49176\.cache\huggingface\datasets\thermostat\multi_nli-bert-lime\1.0.1\0cbe93e1fbe5b8ed0217559442d8b49a80fd4c2787185f2d7940817c67d8707b. Subsequent calls will reuse this data.


In [15]:
print(f'{u_occ.explainer_name} --- same map as in previous example')
u_occ.render()
print(u_intg.explainer_name)
u_intg.render()
print(u_lime.explainer_name)
u_lime.render()

Occlusion --- same map as in previous example


LayerIntegratedGradients


LimeBase


# Rank correlation
A far more interesting, empirical investigation is how well two explainers on the same dataset+model combination correlate regarding their attribution scores.

In [ ]:
imdb_lime = thermostat.load("imdb-bert-lime")
imdb_intg = thermostat.load("imdb-bert-lig")

After loading LIME and LIG explainers for IMDb+BERT, we import the SciPy function that calculates Kendall's tau for rank correlation of attributions.

We merge all attributions to a single list via `.flatten()` which is possible, because accessing the attributions of an entire dataset (Thermopack) via the `.attributions` attribute returns a NumPy array.

In [18]:
from scipy.stats import kendalltau
il_atts = imdb_lime.attributions.flatten()
ig_atts = imdb_intg.attributions.flatten()

kendall_imdb = kendalltau(il_atts, ig_atts)
print(kendall_imdb)

KendalltauResult(correlation=0.025657302000906455, pvalue=0.0)


Let's also consider MultiNLI explanations on BERT plus LIME and LIG.

In [19]:
mnli_lime = thermostat.load("multi_nli-bert-lime")
mnli_intg = thermostat.load("multi_nli-bert-lig")

Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-lime/1.0.0/d4c1fec14831f7d2677ccb8fba33151c9fea8119c4921e647af71cb81299899f)


Loading Thermostat configuration: multi_nli-bert-lime


Reusing dataset thermostat (/home/nfel/.cache/huggingface/datasets/thermostat/multi_nli-bert-lig/1.0.0/d4c1fec14831f7d2677ccb8fba33151c9fea8119c4921e647af71cb81299899f)


Loading Thermostat configuration: multi_nli-bert-lig


In [20]:
ml_atts = mnli_lime.attributions.flatten()
mg_atts = mnli_intg.attributions.flatten()
kendall_mnli = kendalltau(ml_atts, mg_atts)
print(kendall_mnli)

KendalltauResult(correlation=0.10327941961925725, pvalue=0.0)


We find that the correlation between LIG and LIME is higher for MultiNLI than it is for IMDb.
This aligns well with the findings reported in the "Order in the Court" paper by Neely, Schouten et al. (2021): https://api.semanticscholar.org/CorpusID:234096057